Prosta symulacja. 
Jest mapka 50x50
Dajemy w różnych miejscach różne złoża jako agenty - bo się zmieniają.

In [32]:
from random import random

from mesa import Agent
from mesa import Model
import numpy as np
from mesa.space import MultiGrid
from mesa.time import RandomActivationByType

Agents

In [104]:
import MyModel


class Spice(Agent):
    """
    Spice:
    - contains an amount of spice
    - grows 1 amount of spice at each turn
    - max_sugar - the maximum amount of sugar
    """

    def __init__(self, unique_id, model, pos, max_spice):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_spice
        self.max_sugar = max_spice
        
    def step(self):
        """
        Spice grow function, adds one unit of spice at each step until
        max amount
        """
        self.amount = min([self.max_sugar, self.amount+1])


class Sugar(Agent):
    """
    Sugar agent:
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn
    """

    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar
    
    def step(self):
        """
        Sugar grow function, adds one unit of sugar at each step until
        max amount
        """
        self.amount = min([self.max_sugar, self.amount+1])


class Trader(Agent):
    """
    Trader agent:
    - has a metabolism for sugar and spice
    - harvest and traders sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model: MyModel, pos, moore=False, sugar=0, spice=0, metabolism_sugar=0, metabolism_spice=0, vision=1):
        super().__init__(unique_id, model)
        self.moore = moore
        self.pos = pos
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision
        self.isAlive = True
        
    
    def step(self):
        
        pass
    
    def is_occupied_by_other_trader(self, pos):
        """
        helper function of self.move()
        """
        
        if pos == self.pos:
            return False
        
        #contents of each cell in neighborhood
        this_cell = model.grid.get_cell_list_contents(pos)
        for a in this_cell:
            if isinstance(a, Trader):
                return True
        return False
        
    
    def move(self):
        """
        Function for trader agent to identify optimal move for each step in 4 parts
        1 - Identify all possible moves
        2 - Determine which move maximize welfare
        3 - find closes best optiom
        4 - move 
        """
        neighbours = [i for i in model.grid.get_neighborhood(self.pos, self.moore, True, self.vision) if not self.is_occupied_by_other_trader(i)]
        



Model

In [102]:
import mesa
from mesa.time import RandomActivation


class MyModel(Model):
    """
    A model class to manage Sugarspace with Traders
    """
    def __init__(self, width=50, height=50,
                 init_population=200,
                 endowment_min=25,
                 endowment_max=50,
                 metabolism_min=1,
                 metabolism_max=5,
                 vision_min=1,
                 vision_max=5):
        super().__init__()
        print("Its a model")
        self.width = width
        self.height = height
        self.init_population = init_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max        
        
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = mesa.time.RandomActivationByType(self)
        # self.schedule = RandomActivation(self)

        sugar_distribution = np.genfromtxt('sugar-map.txt')
        spice_distribution = np.flip(sugar_distribution, axis=1)    
        # plt.imshow(sugar_distribution, origin='lower')
        # plt.imshow(spice_distribution, origin='lower')
    
        for _, (x,y) in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            max_spice = spice_distribution[x, y]
            
            if max_sugar > 0:
                sugar = Sugar(self.next_id(), self, (x,y), max_sugar)
                # self.grid.place_agent(sugar, (x,y))
                self.schedule.add(sugar)
            if max_spice > 0:
                spice = Sugar(self.next_id(), self, (x,y), max_spice)
                # self.grid.place_agent(spice, (x,y))
                self.schedule.add(spice)
                
        for i in range(self.init_population):
            x = self.random.randint(0, self.width-1)
            y = self.random.randint(0, self.width-1)
            sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            metabolism_sugar: int = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            metabolism_spice  = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
            
            trader = Trader(self.next_id(),
                            self,
                            pos = (x,y),
                            moore = False,
                            sugar=sugar,
                            spice=spice,
                            metabolism_sugar=metabolism_sugar,
                            metabolism_spice=metabolism_spice,
                            vision=vision)
            self.schedule.add(trader)
        
            # print(trader.unique_id, trader.pos, trader.sugar, trader.spice, trader.metabolism_sugar, trader.metabolism_spice, trader.vision)
    
    def step(self):
        """
        Unique step function that does staged activation of sugar and spice
        and then randomly activates traders
        """
        
        for sugar in self.schedule.agents_by_type[Sugar].values():
            sugar.step()
        for spice in self.schedule.agents_by_type[Spice].values():
            spice.step()
        
        #Steo trader agents to account for agent death and removal we need a separate data to iterate.
        traders_shuffle = list(self.schedule.agents_by_type[Trader].values())
        self.random.shuffle(traders_shuffle)
        for agent in traders_shuffle:
            agent.move()
        
        
        self.schedule.steps += 1
        
        
        # automatycznie
        # self.schedule.step()
        
    def run_model(self, step_count=1000):

        for i in range(step_count):
            self.step()


In [100]:
model = MyModel()

Its a model


In [101]:
model.run_model(step_count=5)

1 0
2 0
3 0
4 0
5 0


C:\Users\Mati\AppData\Local\Temp\ipykernel_6972\2181013748.py:79: DeprecationWarning: Because of the shift to using AgentSet, in the future this attribute will return a dict withtype as key as AgentSet as value. Future behavior is available via RandomActivationByType._agents_by_type
  for sugar in self.schedule.agents_by_type[Sugar].values():
C:\Users\Mati\AppData\Local\Temp\ipykernel_6972\2181013748.py:81: DeprecationWarning: Because of the shift to using AgentSet, in the future this attribute will return a dict withtype as key as AgentSet as value. Future behavior is available via RandomActivationByType._agents_by_type
  for spice in self.schedule.agents_by_type[Spice].values():
C:\Users\Mati\AppData\Local\Temp\ipykernel_6972\2181013748.py:85: DeprecationWarning: Because of the shift to using AgentSet, in the future this attribute will return a dict withtype as key as AgentSet as value. Future behavior is available via RandomActivationByType._agents_by_type
  for trader in self.sched